**RAG Implementation**

In [ ]:
!pip install -qU langchain pinecone-client google-generativeai openai tqdm chromadb langchain_community langchain-google-genai --quiet

In [2]:
# Step 2: Import Required Modules
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.vectorstores import Chroma
from langchain.document_loaders import CSVLoader
from langchain.prompts import PromptTemplate

In [4]:
# Step 3: loading the dataset
path = '/content/faq_bot_university.csv'
# changed line
loader = CSVLoader(file_path=path, source_column='Answer')
documents = loader.load()

In [5]:
from google.colab import userdata
key=userdata.get('GOOGLE_API_KEY')

In [6]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [7]:
# Step 4: Initialize Embeddings
embedding_model = GoogleGenerativeAIEmbeddings(google_api_key=key,  model="models/text-embedding-004")

In [8]:
# Step 5: Create a Chroma Vector Store
vector_store = Chroma.from_documents(documents, embedding=embedding_model)

In [9]:
llm = ChatGoogleGenerativeAI(google_api_key=key,
                                   model="gemini-1.5-flash")

In [10]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

print(type(retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [ ]:
sample_docs = retriever.get_relevant_documents("admission requirements")
for doc in sample_docs:
    print(doc.page_content)

In [12]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chat_template = ChatPromptTemplate.from_messages([
    # System Message Prompt Template
    SystemMessage(content="""You are a Helpful Faq Bot.
                  Given a context and question from user,
                  you should answer based on the given context."""),
    # Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""Answer the question based on the given context.
    Context: {context}
    Question: {question}
    Answer: """)
])

output_parser = StrOutputParser()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_template
    | llm
    | output_parser
)

In [13]:
response = rag_chain.invoke("""What courses are offered in Computer Science?""")

response

'Courses include AI, Data Science, and Web Development.'

In [14]:
# Step 7: Set Up Retrieval-Augmented Generation (RAG) Chain
from langchain.chains import RetrievalQA
faq_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever,chain_type="stuff",)

In [16]:
# Step 9: Ask a Question
question = "What courses are offered in Computer Science?"
response = faq_chain.run(question)

print("Response:", response)

Response: Courses include AI, Data Science, and Web Development.
